In [2]:
from math import log
import numpy as np
import operator
import matplotlib.pyplot as plt

In [3]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 侮辱性文字, 0 正常言论
    return postingList,classVec

In [4]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

In [6]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [29]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [36]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)      #change to ones() 
    p0Denom = 0.0; 
    p1Denom = 0.0                  #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = (p1Num/p1Denom)          #change to log()
    p0Vect = (p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [37]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
trainNB0(trainMat, listClasses)

(array([0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.125     ,
        0.04166667, 0.        , 0.        , 0.04166667, 0.        ,
        0.        , 0.04166667, 0.        , 0.08333333, 0.        ,
        0.        , 0.04166667, 0.04166667, 0.04166667, 0.        ,
        0.        , 0.04166667, 0.        , 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.05263158, 0.05263158, 0.        , 0.05263158,
        0.05263158, 0.        , 0.05263158, 0.05263158, 0.15789474,
        0.05263158, 0.        , 0.10526316, 0.        , 0.05263158,
        0.05263158, 0.05263158, 0.05263158, 0.        , 0.        ,
        0.05263158, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.10526316]),
 0.5)

In [52]:
def trainNB1(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)      #change to ones() 
    p0Denom = 2.0; 
    p1Denom = 2.0                  #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)          #change to log()
    p0Vect = np.log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [53]:
p0V, p1V, pAb= trainNB1(trainMat, listClasses)

In [54]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

In [61]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB1(np.array(trainMat),np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

In [62]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [63]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [230]:
def textParse(bigString):    #input is big string, #output is word list
    #print ('The string is: ',bigString)
    import re
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 4] 

In [164]:
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        #print(i, end='')
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50));
    testSet=[]           #create test set
    for i in range(10):
        randIndex = int(np.random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
        
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
        
    p0V,p1V,pSpam = trainNB1(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            #print ("classification error",docList[docIndex])
    print ('Test count %d error count %d rate is: %.2f'%(len(testSet), errorCount,float(errorCount)/len(testSet) * 100))

In [165]:
for i in range(25):
    spamTest()

Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 2 rate is: 20.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error cou

In [199]:
import feedparser
ny = feedparser.parse('http://www.nasa.gov/rss/dyn/image_of_the_day.rss')
ny['entries']

[{'title': "Ringing in InSight's Landing on the Red Planet",
  'title_detail': {'type': 'text/plain',
   'language': 'en',
   'base': 'http://www.nasa.gov/',
   'value': "Ringing in InSight's Landing on the Red Planet"},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'http://www.nasa.gov/image-feature/ringing-in-insights-landing-on-the-red-planet'},
   {'length': '1609963',
    'type': 'image/jpeg',
    'href': 'http://www.nasa.gov/sites/default/files/thumbnails/image/31126336767_e68c3ac5bb_k.jpg',
    'rel': 'enclosure'}],
  'link': 'http://www.nasa.gov/image-feature/ringing-in-insights-landing-on-the-red-planet',
  'summary': "NASA Deputy Associate Administrator Melanie Saunders and astronaut Mike Massimino ring the closing bell of Nasdaq after the NASA's InSight lander successfully touched down on the surface of Mars.",
  'summary_detail': {'type': 'text/html',
   'language': 'en',
   'base': 'http://www.nasa.gov/',
   'value': "NASA Deputy Associate Administr

In [200]:
sf = feedparser.parse('https://www.craigslist.org/about/best/all/index.rss') 
#'http://sports.yahoo.com/nba/teams/hou/rss.xml')
sf['entries']

[{'id': 'https://www.craigslist.org/about/best/sfo/6725919483.html',
  'title': 'Apple Macintosh SE STONE CASTING',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://www.craigslist.org/about/best/all/index.rss',
   'value': 'Apple Macintosh SE STONE CASTING'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://www.craigslist.org/about/best/sfo/6725919483.html'}],
  'link': 'https://www.craigslist.org/about/best/sfo/6725919483.html',
  'summary': "Apple Macintosh 512/Plus/SE Stone Casting<br>\n<br>\nNo my old  Macintosh SE didn't turn to stone when it saw me wearing an Apple Watch.<br>\n<br>\nThis is a high quality casting of an early 1980's classic. I can't id the material but it looks a lot like concrete. I just haven't seen concrete castings of this quality or showing this much detail. Truly a one of a kind piece for the hard core Macintosh fan/evangelist. Don't let this one slip past you.You may not see another one.<br>\n<b

In [249]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:15]    

In [250]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
   
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    #print (top30Words)
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = list(range(2*minLen)) 
    testSet=[]           #create test set
    for i in range(20):
        randIndex = int(np.random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; 
    trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB1(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print ('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

In [257]:
vocabList, pSF, pNY = localWords(ny,sf)

the error rate is:  0.6


In [266]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -5.5 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -4.5 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print ("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print (item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print ("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print (item[0])

In [267]:
getTopWords(ny, sf)

the error rate is:  0.45
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
collection
looking
money
someone
could
thing
think
while
never
should
tapes
video
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
earth
image
pilot
telescope
astronaut
cygnus
captured
today
nebula
michael
insight
collins
river
planets
spitzer
international
stars
round
station
filled
grumman
northrop
mission
